# GROUP BY and JOIN Clauses

In this chapter, we continue with the SQL SELECT statement by learning the GROUP BY and JOIN clauses. Let's recreate the connection string and read in all of the tables as pandas DataFrames from the healthcare database again.

In [ ]:
import pandas as pd
CS = 'sqlite:///../data/databases/healthcare.db'
patient = pd.read_sql('patient', CS)
doctor = pd.read_sql('doctor', CS)
clinic = pd.read_sql('clinic', CS)
procedure = pd.read_sql('procedure', CS)
appointment = pd.read_sql('appointment', CS)

## The GROUP BY clause

The GROUP BY clause works similarly to the `groupby` DataFrame method. Unique values in the provided column form independent groups. Like pandas, an aggregating function must be used to return a single result for each group. The same components (**grouping column**, **aggregating column**, and **aggregating function**) are used with the GROUP BY clause. Here, the grouping column is `major_category`, there is no aggregating column as we are just finding the size of each group, and the aggregating function is `count`. Note that the grouping columns must appear in the SELECT clause.

In [ ]:
sql = """
SELECT major_category, count(*) AS size
FROM procedure
GROUP BY major_category
"""
pd.read_sql(sql, CS)

Replicate in pandas with the `groupby` `size` method.

In [ ]:
procedure.groupby('major_category').size()

Any number of aggregations can be done simultaneously for each group. Here, we compute summary statistics on cost. The `round` function is used to round the average cost to the nearest 10 cents. You may be wondering why the `round` function can be used when its not an aggregating function. It is applied after the `avg` function has taken the aggregation.

In [ ]:
sql = """
SELECT major_category, 
       count(*) AS size, 
       count(cost) AS non_null_cost,
       min(cost) AS min_cost,
       max(cost) AS max_cost,
       round(avg(cost), 1) AS avg_cost,
       sum(cost) AS total_cost
FROM procedure
GROUP BY major_category
"""
pd.read_sql(sql, CS)

In pandas, rename the aggregated columns within the `agg` method.

In [ ]:
(procedure.groupby('major_category')['cost']
          .agg(size='size', 
               non_null_cost='count', 
               min_cost='min',
               max_cost='max', 
               avg_cost='mean', 
               total_cost='sum')
          .round({'avg_cost': 1})
          .reset_index())

### Grouping by multiple columns

Separate columns after the GROUP BY clause to group by multiple columns. Here, unique combinations of `major_category` and `minor_category` form groups. We also aggregate both `cost` and `procedure_id`.

In [ ]:
sql = """
SELECT major_category, minor_category,
       count(cost) AS non_null_cost,
       round(avg(cost), 1) AS avg_cost,
       min(procedure_id) as min_procedure_id,
       max(procedure_id) as max_procedure_id
FROM procedure
GROUP BY major_category, minor_category
LIMIT 3
"""
pd.read_sql(sql, CS)

With pandas, the grouping columns are placed in a list and the `agg` method must now use a tuple to specify both the aggregating column and aggregating function.

In [ ]:
(procedure.groupby(['major_category', 'minor_category'])
          .agg(non_null_cost=('cost', 'count'),
               avg_cost=('cost', 'mean'),
               min_procedure_id=('procedure_id', 'min'),
               max_procedure_id=('procedure_id', 'max'))
          .round({'avg_cost': 1})
          .reset_index()
          .head(3))

### Filtering before grouping

The WHERE clause filters the data before it is grouped. Here, we filter out all the rows with cost less than 300 and then perform the same grouping.

In [ ]:
sql = """
SELECT major_category, minor_category,
       count(cost) AS non_null_cost,
       round(avg(cost), 1) AS avg_cost,
       min(procedure_id) as min_procedure_id,
       max(procedure_id) as max_procedure_id
FROM procedure
WHERE cost < 300
GROUP BY major_category, minor_category
LIMIT 3
"""
pd.read_sql(sql, CS)

With pandas, use the `query` method before calling `groupby`.

In [ ]:
(procedure.query('cost < 300')
          .groupby(['major_category', 'minor_category'])
          .agg(non_null_cost=('cost', 'count'),
               avg_cost=('cost', 'mean'),
               min_procedure_id=('procedure_id', 'min'),
               max_procedure_id=('procedure_id', 'max'))
          .round({'avg_cost': 1})
          .reset_index()
          .head(3))

### Filtering after grouping with the HAVING subclause

The HAVING subclause cannot be used on its own and must immediately proceed the GROUP BY clause. Place a boolean condition to the right of it using the columns created from the grouping. Here, we filter for groups that have an average cost between 620 and 640. The WHERE and HAVING clauses are very similar - the former filters the data before the grouping and the latter after.

In [ ]:
sql = """
SELECT major_category, minor_category,
       count(cost) AS non_null_cost,
       round(avg(cost), 1) AS avg_cost,
       min(procedure_id) as min_procedure_id,
       max(procedure_id) as max_procedure_id
FROM procedure
GROUP BY major_category, minor_category
HAVING avg_cost BETWEEN 620 AND 640
"""
pd.read_sql(sql, CS)

Replicate by calling the `query` method after the `groupby` method in pandas.

In [ ]:
(procedure.groupby(['major_category', 'minor_category'])
          .agg(non_null_cost=('cost', 'count'),
               avg_cost=('cost', 'mean'),
               min_procedure_id=('procedure_id', 'min'),
               max_procedure_id=('procedure_id', 'max'))
          .round({'avg_cost': 1})
          .query('620 <= avg_cost <= 640')
          .reset_index())

You can filter both before and after the grouping by providing WHERE and HAVING clauses. Here, we filter for procedures with a major category of either "Surgery" or "Radiology" and cost of less than 300. After grouping, we filter for groups with more than 50 records.

In [ ]:
sql = """
SELECT major_category, minor_category,
       count(cost) AS non_null_cost,
       round(avg(cost), 1) AS avg_cost,
       min(procedure_id) as min_procedure_id,
       max(procedure_id) as max_procedure_id
FROM procedure 
WHERE major_category in ("Surgery", "Radiology") and cost < 300
GROUP BY major_category, minor_category
HAVING non_null_cost > 50
"""
pd.read_sql(sql, CS)

With pandas, call the `query` method before and after the `groupby` method.

In [ ]:
(procedure.query('major_category in ("Surgery", "Radiology") and cost < 300')
          .groupby(['major_category', 'minor_category'])
          .agg(non_null_cost=('cost', 'count'),
               avg_cost=('cost', 'mean'),
               min_procedure_id=('procedure_id', 'min'),
               max_procedure_id=('procedure_id', 'max'))
          .round({'avg_cost': 1})
          .query('non_null_cost > 50')
          .reset_index())

## Ordering after grouping

The ORDER BY clause executes after the GROUP BY and HAVING clauses have finished aggregating and filtering the data. Here, we find all groups with an average cost above 600 and then return the ones with the most known procedure costs.

In [ ]:
sql = """
SELECT major_category, minor_category,
       count(cost) AS non_null_cost,
       round(avg(cost), 1) AS avg_cost,
       max(procedure_id) as max_procedure_id
FROM procedure
GROUP BY major_category, minor_category
HAVING avg_cost > 600
ORDER BY non_null_cost DESC
LIMIT 5
"""
pd.read_sql(sql, CS)

With pandas, call `sort_values` after the `groupby` and `query `methods.

In [ ]:
(procedure.groupby(['major_category', 'minor_category'])
          .agg(non_null_cost=('cost', 'count'),
               avg_cost=('cost', 'mean'),
               max_procedure_id=('procedure_id', 'max'))
          .round({'avg_cost': 1})
          .query('avg_cost > 600')
          .sort_values('non_null_cost', ascending=False)
          .reset_index()
          .head(5))

### Grouping by calculated columns

It's possible to group by columns not directly in the table. Here, we use the modulus operator to find the last digit of each `procedure_id` and use it as the grouping column. We then find the average cost and count of this group.

In [ ]:
sql = """
SELECT 
    procedure_id % 10 as last_procedure_id_digit, 
    round(avg(cost), 1) as avg_cost, 
    count(*) as ct
FROM procedure
GROUP BY last_procedure_id_digit

"""
pd.read_sql(sql, CS)

In pandas, the special grouping column must be created as a separate Series beforehand. Setting the `name` attribute of the Series will make it so that it used as the column name after the grouping.

In [ ]:
s = procedure['procedure_id'] % 10
s.name = 'last_procedure_id_digit'
(procedure.groupby(s)['cost']
          .agg(avg_cost='mean',
               ct='count')
          .round({'avg_cost': 1})
          .reset_index())

## The JOIN subclause

The JOIN subclause must appear directly after the FROM clause and is used to join the rows of two tables together based on a boolean condition. The generic syntax is as follows:

`FROM left_table AS a JOIN right_table AS b ON a.column = b.column`

Each table is provided an **alias** with AS which allows us to reference that table with the alias instead of the entire table name. This helps shorten the syntax. The condition on how the rows are joined is provided after the ON keyword. The simplest case is when a single value in the left table's row is equal to a single value in the right table's row.

It's important to have the database diagram available to understand how to join tables. The diagram will show you which columns join with which tables. As discussed in the Joining Data part, there are three major types of joins, an inner join, a left join, and a full join. Inner joins keep rows where the condition after the ON keyword evaluates as true. An inner join occurs by default when using the keyword JOIN, but you can use INNER JOIN for clarity. 

Below, we create an inner join between the appointment table and doctor table joining them on column `doctor_id`. From the database diagram, each row in the appointment table should align with exactly one row in the doctor table.

In [ ]:
sql = """
SELECT *
FROM appointment AS a
    INNER JOIN doctor AS d ON a.doctor_id = d.doctor_id
LIMIT 3
"""
pd.read_sql(sql, CS)

Note that the alias `a` refers to the appointment table and that the alias `d` refers to the doctor table. Aliases can be any length are are typically one or just a few characters in length. Aliases are not necessary and the statement could be rewritten without them to be:

```sql
SELECT *
FROM appointment
    INNER JOIN doctor ON appointment.doctor_id = doctor.doctor_id
```

The `merge` method duplicates this join. Provide it the joining column and the type of join. The result does not preserve the original order of the left table like in SQL so the `sort_values` method is called to revert it to its original ordering.

In [ ]:
(appointment.merge(doctor, on='doctor_id', how='inner')
           .sort_values('appointment_id').head(3))

### Use aliases to select specific columns

Notice the SQL result has the `doctor_id` column twice, while with pandas, it appears once. Since we used `SELECT *`, all of the columns from all of the tables are returned. We can select specific columns from each table using each table's alias. Here, we select all columns from the left table and three of the columns from the right.

In [ ]:
sql = """
SELECT a.*, d.first_name, d.last_name, d.specialty
FROM appointment AS a
    INNER JOIN doctor AS d ON a.doctor_id = d.doctor_id
LIMIT 3
"""
pd.read_sql(sql, CS)

### Other clauses when joining

Joining of the tables happens before any of the other clauses. Here, we filter for a specific clinic and patient after finding the doctor's specialty.

In [ ]:
sql = """
SELECT a.appointment_id, a.clinic_id, a.date, d.specialty
FROM appointment AS a
    INNER JOIN doctor AS d ON a.doctor_id = d.doctor_id
WHERE clinic_id = 4 and patient_id = 100
LIMIT 3
"""
pd.read_sql(sql, CS)

In pandas, use the `query` method after the `merge` method.

In [ ]:
(appointment.merge(doctor, on='doctor_id', how='inner')
           .sort_values('appointment_id')
           .query('clinic_id == 4 and patient_id == 100')
           [['appointment_id', 'clinic_id', 'date', 'specialty']]
            .head(3))

The number of appointments by doctor specialty can be retrieved by grouping after joining the tables together.

In [ ]:
sql = """
SELECT d.specialty, count(*) as num_appointments
FROM appointment AS a
    INNER JOIN doctor AS d ON a.doctor_id = d.doctor_id
GROUP BY d.specialty
"""
pd.read_sql(sql, CS)

In pandas, we call the `groupby` method after `merge`.

In [ ]:
(appointment.merge(doctor, on='doctor_id', how='inner')
            .groupby('specialty').size())

Here, we find the number of appointments by specialty in clinics 3 and 4. We then sort the results from greatest to least.

In [ ]:
sql = """
SELECT d.specialty, count(*) as num_appointments
FROM appointment AS a
    INNER JOIN doctor AS d ON a.doctor_id = d.doctor_id
WHERE clinic_id in (3, 4)
GROUP BY d.specialty
ORDER BY num_appointments DESC
"""
pd.read_sql(sql, CS)

Add the `query` method after `merge` and then use `sort_values` as the last command.

In [ ]:
(appointment.merge(doctor, on='doctor_id', how='inner')
            .query('clinic_id in (3, 4)')
            .groupby('specialty').size()
            .sort_values(ascending=False))

### Joining more than two tables

Any number of tables may be joined together by adding more JOIN subclauses. Here, we join the three other tables to the appointment table selecting one or two columns from each table. Each row has four equality conditions that must be satisfied.

In [ ]:
sql = """
SELECT a.appointment_id,
       a.date,
       d.specialty AS doctor_specialty, 
       c.name AS clinic_name,
       p.first_name as patient_first_name,
       pr.major_category,
       pr.cost      
FROM appointment AS a
    INNER JOIN doctor AS d ON a.doctor_id = d.doctor_id
    INNER JOIN clinic AS c ON a.clinic_id = c.clinic_id
    INNER JOIN patient AS p ON a.patient_id = p.patient_id
    INNER JOIN procedure AS pr ON a.procedure_id = pr.procedure_id
LIMIT 3
"""
pd.read_sql(sql, CS)

A similar result in pandas can be achieved by first selecting the desired columns for each table as separate DataFrames and then passing them to the `merge` method. The appointment table needs to be have all of its foreign key columns before calling `merge`. They are then dropped after all the joins have completed. 

In [ ]:
d = doctor[['doctor_id', 'specialty']]
c = clinic[['clinic_id', 'name']]
p = patient[['patient_id', 'first_name']]
pr = procedure[['procedure_id', 'major_category', 'cost']]
(appointment.merge(d, on='doctor_id')
            .merge(c, on='clinic_id')
            .merge(p, on='patient_id')
            .merge(pr, on='procedure_id')
            .drop(columns = ['patient_id', 'clinic_id', 'doctor_id', 'procedure_id'])
            .head(3))

This example highlights the major differences between a declarative language like SQL and a procedural language like Python. In SQL, the entire statement is read by the execution engine before it runs. It sees what the final result needs to be - the number of tables to join together, the specific columns from each table, etc... 

In Python, every statement runs in the order that it appears. The very first `merge` from above does not know that there will be calls to other `merge` methods below it. SQL has a big advantage, in this instance, by being able to see and understand the entire statement before its execution. In Python, each statement is independent of the next, so no planning can occur. The multiple join statement above runs much faster in SQL than in Python.

### Left joins

In addition to all of the rows of an inner join, a left join includes all rows in the left table that have no matches in the right table. All of these rows without a match will have NULL as the value for the columns in the right table.

A few of the doctors in the doctor table do not appear in the appointment table. Let's perform a left join on the doctor's table with the appointment and assign the result to the variable name `df`. 

In [ ]:
sql = """
SELECT *
FROM doctor AS d
    LEFT JOIN appointment AS a ON d.doctor_id = a.doctor_id
"""
df = pd.read_sql(sql, CS)
df.head(3)

This results in 9,124 rows.

In [ ]:
len(df)

But the appointment table has three less rows in it.

In [ ]:
len(appointment)

We can find the three doctors who do not appear appear in the appointment table by filtering for them in the WHERE clause. We test that the doctor_id column in the appointment table is NULL, and therefore did not have a match. A left join keeps these doctors in the result, even though their doctor_id does not appear in the appointment table.

In [ ]:
sql = """
SELECT *
FROM doctor AS d
    LEFT JOIN appointment AS a ON d.doctor_id = a.doctor_id
WHERE a.doctor_id is NULL
"""
df = pd.read_sql(sql, CS)
df.head(3)

These three doctors would not appear if an INNER JOIN were used. Here, we change the type of join to confirm the result.

In [ ]:
sql = """
SELECT *
FROM doctor AS d
    INNER JOIN appointment AS a ON d.doctor_id = a.doctor_id
WHERE a.doctor_id is NULL
"""
df = pd.read_sql(sql, CS)
df.head(3)

Set the `how` parameter to `'left'` in pandas and then filter using boolean selection to find the rows that have missing values for the appointment table columns. You'll discover the same doctors absent from the appointment table.

In [ ]:
df = doctor.merge(appointment, on='doctor_id', how='left')
filt = df['appointment_id'].isna()
df[filt]

## Right and Full joins

Unfortunately, SQLite does not provide right or full joins like most other RDBMS's. A right join can mostly be replicated by switching the order of the joining tables and performing a left join. A full join requires the use of the UNION clause, which stacks unique records of tables one on top of the other similar to the `pd.concat` function.

## Exercises

All of these exercises use the Chinook database with SQL SELECT statements to answer each of the following exercises.

### Exercise 1

<span style="color:green; font-size:16px">Find the grand total of invoices by billingcountry. Order the results by this total from greatest to least.</span>

### Exercise 2

<span style="color:green; font-size:16px">Find the count, min, max, and avg milliseconds by genreid in the tracks table. Convert the milliseconds to minutes. Order the results by count descending.</span>

### Exercise 3

<span style="color:green; font-size:16px">Using the invoice_items table, count the times each trackid and invoiceid combination appear. This number should be one, as an invoice should not have multiple instances of the same trackid. Can you verify that there is in fact at most one?</span>

### Exercise 4

<span style="color:green; font-size:16px">Calculate the total revenue of each track using the invoice_items table. Return the top five tracks by revenue.</span>

### Exercise 5

<span style="color:green; font-size:16px">Create a table with the track name and genre name (not genreid).</span>

### Exercise 6

<span style="color:green; font-size:16px">Create a table with the track name, genre name, album title, and artist title. You will need to join four tables together.</span>

### Exercise 7

<span style="color:green; font-size:16px">For tracks less than two minutes in length, count the occurrence of each media type. Make sure to use the media type name.</span>